In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load the pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)

# Freeze only the first few convolutional layers
for i, param in enumerate(vgg16.features[:10]):  # Freeze the first 10 layers
    param.requires_grad = False

# Modify the classifier part for binary classification
vgg16.classifier[6] = nn.Linear(4096, 1)  # Replace the last layer with a binary classifier (1 output for binary)
vgg16.classifier.add_module('sigmoid', nn.Sigmoid())  # Add a sigmoid activation for binary classification

# Check which parameters are trainable
for name, param in vgg16.named_parameters():
    if param.requires_grad:
        print(f"{name} is trainable")

# Example of moving the model to a GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)

# Modify the input layer for 32x32x3 input
vgg16.features[0] = nn.Conv2d(3, 64, kernel_size=3, padding=1)

# Freeze the first few convolutional layers (e.g., first 10 layers)
for i, param in enumerate(vgg16.features[:10]):  # Freeze the first 10 layers
    param.requires_grad = False

# Modify the classifier part for the new input size after flattening
vgg16.classifier = nn.Sequential(
    nn.Linear(512 * 1 * 1, 4096),  # Adjust for 32x32 input
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(4096, 1),  # Binary classification output
    nn.Sigmoid()  # Sigmoid for binary classification
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)

# Print trainable layers (for verification)
for name, param in vgg16.named_parameters():
    if param.requires_grad:
        print(f"{name} is trainable")


In [ ]:
import torch.nn as nn

# Binary classification, so no need for num_classes
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for binary classification
criterion = criterion.to(device)  # Move to device (GPU if available)


In [ ]:
import torch.optim as optim

# Define the hyperparameters
learning_rate = 0.001  # η
beta1 = 0.9  # β1
beta2 = 0.999  # β2
epsilon = 1e-7  # ε

# Create an instance of the Adam optimizer, but only pass the trainable parameters
optimizer = optim.Adam(filter(lambda p: p.requires_grad, vgg16.parameters()), 
                       lr=learning_rate, betas=(beta1, beta2), eps=epsilon)


In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Load your pretrained model weights here if needed
model.load_state_dict(torch.load('Adam_weights.pth'))

# Create empty lists to store training and validation losses and accuracies
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Set the current epoch number to continue from where you left off
current_epoch = 0  # Change this to the epoch you want to continue from

num_epochs = 2  # Set the total number of epochs you want to run

# Define early stopping parameters
patience = 5  # Number of epochs with no improvement after which training will stop
min_delta = 0.001  # Minimum change in validation loss to be considered as improvement
best_val_loss = float('inf')
epochs_without_improvement = 0

# Continue training for additional epochs
for epoch in range(current_epoch, num_epochs):
    model.train()
    running_train_loss = 0.0
    train_preds = []
    train_targets = []

    # Training loop with a progress bar
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} (Training)'):
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels.float())  # Convert labels to float for BCEWithLogitsLoss
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

        # Convert logits to probabilities with sigmoid
        probs = torch.sigmoid(outputs)
        predicted = (probs > 0.5).float()  # Convert probabilities to binary (0 or 1)
        
        train_preds.extend(predicted.cpu().numpy())
        train_targets.extend(labels.cpu().numpy())

    # Calculate and print the average training loss for this epoch
    avg_train_loss = running_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Calculate training accuracy for this epoch
    train_accuracy = accuracy_score(train_targets, train_preds)
    train_accuracies.append(train_accuracy)

    model.eval()  # Set the model to evaluation mode
    running_val_loss = 0.0
    val_preds = []
    val_targets = []

    # Validation loop with a progress bar
    for images, labels in tqdm(val_loader, desc=f'Epoch {epoch + 1}/{num_epochs} (Validation)'):
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        with torch.no_grad():
            outputs = model(images)
            loss = criterion(outputs, labels.float())  # Use float labels for BCEWithLogitsLoss
            running_val_loss += loss.item()

            # Convert logits to probabilities with sigmoid
            probs = torch.sigmoid(outputs)
            predicted = (probs > 0.5).float()  # Convert probabilities to binary (0 or 1)

            val_preds.extend(predicted.cpu().numpy())
            val_targets.extend(labels.cpu().numpy())

    # Calculate and print the average validation loss for this epoch
    avg_val_loss = running_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    
    # Calculate validation accuracy for this epoch
    val_accuracy = accuracy_score(val_targets, val_preds)
    val_accuracies.append(val_accuracy)

    # Early stopping
    if avg_val_loss + min_delta < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), 'Adam_weights.pth')
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f'Early stopping after {patience} epochs without improvement.')
            break

    # Print loss and accuracy for this epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}')
    print(f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

print('Finished Training')


In [ ]:
#Training vs Validation accuracies
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Train_Accuracy', marker='o')
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Val_Accuracy', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training vs. Validation Accuracies')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
#Training vs Validation loss
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(range(1, num_epochs + 1), train_losses, label='Train_Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Val_Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Loss')
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, roc_curve, confusion_matrix, classification_report, f1_score, recall_score

# Initialize empty lists to store true labels and predicted class probabilities
true_labels = []
predicted_probs = []

# Set the model to evaluation mode
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)  # Move images to GPU
        labels = labels.to(device)  # Move labels to GPU (optional, depending on usage)
        outputs = model(images)

        # Get the predicted class probabilities using sigmoid
        probs = torch.sigmoid(outputs).cpu().numpy().flatten()  # Convert logits to probabilities
        predicted_probs.extend(probs)

        # Store true labels
        true_labels.extend(labels.cpu().numpy())

# Convert probabilities to binary labels (0 or 1) using a threshold of 0.5
predicted_labels = [1 if prob >= 0.5 else 0 for prob in predicted_probs]

# Calculate the accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate the AUC
roc_auc = roc_auc_score(true_labels, predicted_probs)

# Calculate precision, recall, and F1 score
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# Calculate sensitivity (True Positive Rate)
conf_matrix = confusion_matrix(true_labels, predicted_labels)
sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])

# Calculate specificity (True Negative Rate)
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])

# Print the results
print(f'AUC: {roc_auc:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall (Sensitivity): {recall:.4f}')
print(f'Specificity: {specificity:.4f}')
print(f'F1 Score: {f1:.4f}')


In [ ]:
import matplotlib.pyplot as plt

# ROC Curve
fpr, tpr, thresholds = roc_curve(true_labels, predicted_probs)
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Access values from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]
TP = conf_matrix[1, 1]

# Now you can use these values as needed

# Define the confusion matrix
conf_matrix = np.array([[TN, FP], [FN, TP]])  

# Confusion Matrix
plt.figure(figsize=(6, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)

# Annotate cells with their values
thresh = conf_matrix.max() / 2.0  # Set a threshold for text coloring
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment="center",
                 color="white" if conf_matrix[i, j] > thresh else "black")

plt.title('Confusion Matrix')
plt.colorbar()
plt.xticks([0, 1], ['Predicted 0', 'Predicted 1'])
plt.yticks([0, 1], ['Actual 0', 'Actual 1'])
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()
